In [0]:
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD,Adam

from sklearn.utils import shuffle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [14]:
!wget -nc http://files.grouplens.org/datasets/movielens/ml-20m.zip #uso uno con 20M ratings. 

File ‘ml-20m.zip’ already there; not retrieving.



In [15]:
!unzip -n ml-20m.zip #quiero tatings.csv

Archive:  ml-20m.zip


In [16]:
df = pd.read_csv('ml-20m/ratings.csv')
df.head()

userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819
3       1       47     3.5  1112484727
4       1       50     3.5  1112484580

In [0]:
#no puedo asumir que esta formateado como quiero. No quiero integers faltantes en ids. 

df.userId = pd.Categorical(df.userId)
df['new_user_id'] = df.userId.cat.codes #asigna categorical encodign a esos integers

df.movieId = pd.Categorical(df.movieId)
df['new_movie_id'] = df.movieId.cat.codes

In [30]:
#cambio todo a numpy arrays
user_ids=df['new_user_id'].values
movie_ids=df['new_movie_id'].values
ratings = df['rating'].values

print(len(movie_ids))
print(len(user_ids))

20000263
20000263


In [7]:
N = len(set(user_ids))
M = len(set(movie_ids))
K = 10 #imbeding dimmension

print(N)

138493


In [0]:
#armo el modelo

u = Input(shape=(1,1))
uEmb = Embedding(N,K)(u)

m = Input(shape=(1,1))
mEmb = Embedding(M,K)(m)

uEmb = Flatten()(uEmb)
mEmb = Flatten()(mEmb)

x = Concatenate()([uEmb,mEmb])

x = Dense(1024, activation='relu')(x)
x = Dense(1)(x)

In [0]:
model = Model(inputs=[u,m], outputs=x)
model.compile(loss='mse',optimizer=SGD(lr=0.08,momentum=0.9))

In [10]:
user_ids,movie_ids,ratings = shuffle(user_ids,movie_ids,ratings)
Ntrain = int(0.8*len(ratings))
train_user = user_ids[:Ntrain]
train_movie = movie_ids[:Ntrain]
train_ratings = ratings[:Ntrain]

test_user = user_ids[Ntrain:]
test_movie = movie_ids[Ntrain:]
test_ratings = ratings[Ntrain:]

#centreo los ratings. No divido por el std poruqe no me quiero complicar con el scale
avg_rating = train_ratings.mean()
train_ratings = train_ratings - avg_rating 
test_ratings = test_ratings - avg_rating

print(len(test_user))
print(len(test_movie))
print(len(train_ratings))


4000053
4000053
16000210


In [11]:
r = model.fit(
    x=[train_user, train_movie],
    y=train_ratings,
    epochs=25,
    batch_size=1024,
    validation_data=([test_user, test_movie], test_ratings),
)

Epoch 1/25
15626/15626 [==============================] - 92s 6ms/step - loss: 0.7762 - val_loss: 0.7210
Epoch 2/25
15626/15626 [==============================] - 92s 6ms/step - loss: 0.7006 - val_loss: 0.6980
Epoch 3/25
15626/15626 [==============================] - 92s 6ms/step - loss: 0.6785 - val_loss: 0.6837
Epoch 4/25
15626/15626 [==============================] - 91s 6ms/step - loss: 0.6630 - val_loss: 0.6749
Epoch 5/25
15626/15626 [==============================] - 90s 6ms/step - loss: 0.6530 - val_loss: 0.6696
Epoch 6/25
15626/15626 [==============================] - 90s 6ms/step - loss: 0.6426 - val_loss: 0.6612
Epoch 7/25
15626/15626 [==============================] - 91s 6ms/step - loss: 0.6269 - val_loss: 0.6515
Epoch 8/25
15626/15626 [==============================] - 89s 6ms/step - loss: 0.6115 - val_loss: 0.6439
Epoch 9/25
15626/15626 [==============================] - 95s 6ms/step - loss: 0.6007 - val_loss: 0.6497
Epoch 10/25
15626/15626 [==============================

In [54]:
#testeo con algunos 
import math

smallUTest = train_user[:2]
smallMTest = train_movie[:2]

unaPred = [smallUTest,smallMTest]

pred = model.predict(x=unaPred)
print(pred)

rating = math.sqrt(pred[0]**2+pred[1]**2)
print(rating)


[[ 0.61417705]
 [-0.17965771]]
0.6399143277881574
3.5
